### 라이브러리 호출

In [41]:
import requests # url 연결
from bs4 import BeautifulSoup  # html 파싱
import re # html 파싱 이후 정규식으로 이미지 URL 추출
import pandas as pd # 추출된 데이터 데이터프레임 만듦
import os # 이미지 파일 경로 지정
from PIL import Image  # 이미지 저장 라이브러리 PIL(Python Image Library)
from io import BytesIO # 메모리에 있는 바이트 배열을 이진 파일로 다루는 클래스

### 요청 보내서 html 받기

In [2]:
header = {'User-Agent': 'Mozila/5.0 (Windows NT 6.3; Trident/7.0; rv:11.0) like Gecko'}
url = 'https://economist.co.kr/article/list/ecn_SC002001000'
res = requests.get(url, headers= header)
soup = BeautifulSoup(res.text, 'html.parser')


### 선택자 결정 후, 각각의 요소를 리스트에 담음
(웬만하면 하나의 선택자로 다 추출 후 나누는 방식 해봄)

In [3]:
# 선택자
# 기사 제목 :  #articleList > div:nth-child(1) > a > div > h1 > span
# 기사 날짜 : #articleList > div:nth-child(2) > a > span.media-date > span

tmp = []
[tmp.append(i.text) for i in soup.select('#articleList a span')]

#제목은 첫 항이 1이고, 공차가 4인 등차수열
title_list = [i for num, i in enumerate(tmp) if num in range(1,10000,4)] #최대값 1만 하드코딩은 아쉬움

#날짜는 첫 항이 3이고, 공차가 4인 등차수열
post_date = [i for num, i in enumerate(tmp) if num in range(3,10000,4)] #최대값 1만 하드코딩은 아쉬움

del tmp # tmp 삭제해서 메모리 최적화 해봄

### 패턴을 정해서, 이미지 URL을 구함
(정규식 표현법은 추가적으로 탐구해봐야겠음)

In [4]:
pattern = re.compile(r'data-src="([^"]+)"')

# 4의 배수가 URL을 담고 있음
image_url = [pattern.findall(str(i))[0] for num, i in enumerate(soup.select('#articleList a span')) if num % 4 == 0]

### 모든 리스트 합치기
(zip을 써서, 2차원 리스트로 구현)

In [39]:
final_list = []
for x,y,z in zip(title_list, post_date, image_url):
    final_list.append([x,y,z])

final_list[:2]


[['종부세 내는 사람 4.8만명 늘어난 이유 살펴봤더니…’수·다·고’가 대부분',
  '2024-11-30',
  'https://image.economist.co.kr/data/ecn/image/2024/11/30/ecn20241130000003.160x122.0.jpg'],
 ['얼어붙은 부동산 시장…기준금리 인하에도 한동안 ‘겨울바람’ 전망',
  '2024-11-29',
  'https://image.economist.co.kr/data/ecn/image/2024/11/29/ecn20241129000060.160x122.0.jpg']]

### 이미지 저장하기

In [40]:
# 이미지 경로 생성
image_dir = 'images'
if not os.path.exists(image_dir):
    os.makedirs(image_dir)

# 현재 경로에 이미지 경로 추가
image_path = os.path.join(os.getcwd(), image_dir)


# 이미지 url에 요청 보내서 바이트 데이터 가져오기
for num, j in enumerate(image_url): #이미지 이름을 순번으로 저장하기
    tmp_res = requests.get(j, headers=header)
    image = Image.open(BytesIO(tmp_res.content)) # Image 클래스로 바이트 데이터의 이미지 열기
    try :    
        image.save(os.path.join(image_path, str(num) + ".jpg")) #PIL의 File 타입에 대해서 save 함수 쓰기 # title list의 앞 3글자만 따서 저장(왜냐면 특수문자 있을 수 있어서)
    except:
        image.save(os.path.join(image_path, str(num) + ".png")) # 이미지 모드에 RGBA가 있어서 JPG 변환불가 이는 PNG로 처리
        print(f'{num}번에 png 발생 !')







7번에 png 발생 !


### pandas로 데이터 프레임 만들기

In [57]:
df = pd.DataFrame(final_list, columns= ['title', 'post_date', 'image_url'])

df.to_csv('14_project.csv', index = False, encoding='cp949')

pd.read_csv('14_project.csv', encoding='cp949')

,title,post_date,image_url
0,종부세 내는 사람 4.8만명 늘어난 이유 살펴봤더니…’수·다·고’가 대부분,2024-11-30,https://image.economist.co.kr/data/ecn/image/2...
1,얼어붙은 부동산 시장…기준금리 인하에도 한동안 ‘겨울바람’ 전망,2024-11-29,https://image.economist.co.kr/data/ecn/image/2...
2,"이도-한림읍, 청정지역 제주시 한림읍 해양폐기물 처리 ‘맞손’",2024-11-29,https://image.economist.co.kr/data/ecn/image/2...
3,'서울원' 흥행·재무구조 개선에 실적 호조 기록한 HDC현산,2024-11-29,https://image.economist.co.kr/data/ecn/image/2...
4,현대건설 ‘힐스테이트 등촌역’ 견본주택 29일 개관,2024-11-29,https://image.economist.co.kr/data/ecn/image/2...
5,삼쩜삼 “2019년 종부세 과다 납부분 환급 신청 기한 얼마 안 남았어요”,2024-11-28,https://image.economist.co.kr/data/ecn/image/2...
6,'용산국제업무지구' 청사진 공개...초고층 빌딩·글로벌 기업 유치,2024-11-28,https://image.economist.co.kr/data/ecn/image/2...
7,정병윤 리츠협회장 “국내 리츠 경쟁력 높이기 위한 과제 해결 필요”,2024-11-28,https://image.economist.co.kr/data/ecn/image/2...
8,"DL이앤씨, ‘아크로 리츠카운티‘ 분양 예정",2024-11-28,https://image.economist.co.kr/data/ecn/image/2...
9,‘이재명 아파트’도 재건축된다…1기 선도지구 발표,2024-11-27,https://image.economist.co.kr/data/ecn/image/2...
